In [85]:
table_application = """
    CREATE TABLE application (
        application_id int,
        application_number  varchar(10),
        amount int,
        amount_paid int,
        state  varchar(10),
        office_code varchar(10), 
        service_code varchar(10), 
        date_created datetime,
        date_paid datetime,
        date_processed datetime,
        PRIMARY KEY (application_id)
    );
"""

table_office ="""
    CREATE TABLE Office (
        office_code varchar(10),
        office_name varchar(20),
        location_code varchar(10),
        PRIMARY KEY (office_code)
        );
"""

table_location ="""
    CREATE TABLE location (
        location_code varchar(10),
        location_name varchar(20),
        PRIMARY KEY (location_code)
        );
"""

table_service ="""
    CREATE TABLE service (
        service_code varchar(10),
        service_name varchar(20),
        PRIMARY KEY (service_code)
        );
"""

In [86]:
# Create the data base
import sqlite3

DB_FILENAME = 'irembo_application_4.db'

conn = sqlite3.connect(DB_FILENAME)
cursor = conn.cursor()
cursor.execute(table_application)
cursor.execute(table_office)
cursor.execute(table_location)
cursor.execute(table_service)


In [78]:
conn.close()

In [87]:
# --
# Define Office, Location, Application and Service information
# --

office_data = [
    ('O1', 'Office 1', 'L1'),
    ('O2', 'Office 2', 'L2'),
    ('O3', 'Office 3', 'L3'),
    ('O4', 'Office 4', 'L4'),
    ('O5', 'Office 5', 'L5'),
    ('O6', 'Office 6', 'L6'),
    ('O7', 'Office 7', 'L7'),
    ('O8', 'Office 8', 'L8'),
    ('O9', 'Office 9', 'L9'),
    ('O10', 'Office 10', 'L10'),
    ('O11', 'Office 11', 'L11'),
    ('O12', 'Office 12', 'L12'),
    ('O13', 'Office 13', 'L13'),
    ('O14', 'Office 14', 'L14'),
    ('O15', 'Office 15', 'L15'),
    ('O16', 'Office 16', 'L16'),
    ('O17', 'Office 17', 'L17'),
]

location_data = [
    ('L1', 'Location 1'),
    ('L2', 'Location 2'),
    ('L3', 'Location 3'),
    ('L4', 'Location 4'),
    ('L5', 'Location 5'),
    ('L6', 'Location 6'),
    ('L7', 'Location 7'),
    ('L8', 'Location 8'),
]

service_data = [
    ('S1', 'Service 1'),
    ('S2', 'Service 2'),
    ('S3', 'Service 3'),
    ('S4', 'Service 4'),
    ('S5', 'Service 5'),
    ('S6', 'Service 6'),
    ('S7', 'Service 7'),
    ('S8', 'Service 8'),
]

conn = sqlite3.connect(DB_FILENAME)
cursor = conn.cursor()
cursor.executemany('INSERT INTO Office VALUES (?,?,?)', office_data)
cursor.executemany('INSERT INTO Location VALUES (?,?)', location_data)
cursor.executemany('INSERT INTO Service VALUES (?,?)', service_data)
conn.commit()
conn.close()


In [88]:
import string
import random
from datetime import datetime, timedelta

states = ['APPROVED','REJECTED','PENDING_PAYMENT', 'PAID']
prices = [1000, 10000,25000, 20000, 0]

# or a function
def gen_datetime(min_year=2021, max_year=datetime.now().year):
    # generate a datetime in format yyyy-mm-dd hh:mm:ss.000000

    today_datetime = datetime.now()
    return today_datetime - timedelta(days=random.randint(1,1100), hours=random.randint(1,23), minutes=random.randint(1,60), seconds=random.randint(1,60))
    # start = datetime(min_year, 1, 1, 00, 00, 00)
    # years = max_year - min_year + 1
    # end = start + timedelta(days=365 * years)
    # return start + (end - start) * random.random()

def generate_random_application(states=states):
    N = 8
    application_data = []
    strformat = '%Y-%m-%d %H:%M:%S'
    for i in range(100000,150000):
        creationdate = gen_datetime()
        price = prices[random.randint(0,4)]
        application = (
            i+1,
            'A0' + ''.join(random.choices(string.ascii_uppercase + string.digits, k=8)), 
            price, 
            price, 
            states[random.randint(0,3)],
            'O' +''.join(str(random.randint(1,8))),
            'S' +''.join(str(random.randint(1,8))),
            creationdate.strftime(strformat),
            (creationdate + timedelta(hours=9)).strftime(strformat),
            (creationdate + timedelta(hours=24)).strftime(strformat),
        )
        
        application_data.append(application)
    return application_data    

application_data = generate_random_application()

conn = sqlite3.connect(DB_FILENAME)
cursor = conn.cursor()
cursor.executemany('INSERT INTO Application VALUES (?,?,?,?,?,?,?,?,?,?)', application_data)
conn.commit()
conn.close()


In [81]:
def run_query(query=''):
    conn = sqlite3.connect(DB_FILENAME)
    cursor = conn.cursor()
    cursor.execute(query)      
    data = cursor.fetchall()
    print(data)
    conn.close




query_trend = """
SELECT 
    strftime('%Y-%m-%d', date_created) AS application_date,
    COUNT(*) AS approved_applications
FROM 
    application
JOIN 
    Office ON application.office_code = Office.office_code
JOIN 
    location ON Office.location_code = location.location_code
WHERE 
    application.state = 'APPROVED'
    AND location.location_name = 'Location 1'
    AND date_created >= date('now', '-9 days') -- last 10 days including today
    AND date_created <= date('now') -- up to today
GROUP BY 
    strftime('%Y-%m-%d', date_created)
ORDER BY 
    strftime('%Y-%m-%d', date_created) ASC;
"""

run_query(query=query_trend)


[('2024-04-20', 1), ('2024-04-21', 1), ('2024-04-25', 1)]


In [82]:
query = """
SELECT 
    strftime('%Y-%m', date_paid) AS month,
    COUNT(*) AS approved_applications
FROM 
    application
WHERE 
    amount_paid = amount AND state='APPROVED' AND office_code IN (
        SELECT 
            o.office_code
        FROM 
            Office o, location l
        WHERE 
            o.location_code=l.location_code AND l.location_name='Location 2'
    )
GROUP BY 
    month
ORDER BY 
    month;
"""

run_query(query=query)

[('2021-01', 27), ('2021-02', 31), ('2021-03', 30), ('2021-04', 33), ('2021-05', 40), ('2021-06', 28), ('2021-07', 27), ('2021-08', 30), ('2021-09', 31), ('2021-10', 35), ('2021-11', 26), ('2021-12', 33), ('2022-01', 32), ('2022-02', 34), ('2022-03', 39), ('2022-04', 39), ('2022-05', 30), ('2022-06', 29), ('2022-07', 35), ('2022-08', 31), ('2022-09', 37), ('2022-10', 31), ('2022-11', 31), ('2022-12', 22), ('2023-01', 33), ('2023-02', 35), ('2023-03', 34), ('2023-04', 35), ('2023-05', 28), ('2023-06', 32), ('2023-07', 26), ('2023-08', 30), ('2023-09', 26), ('2023-10', 36), ('2023-11', 37), ('2023-12', 39), ('2024-01', 39), ('2024-02', 33), ('2024-03', 29), ('2024-04', 28), ('2024-05', 37), ('2024-06', 33), ('2024-07', 33), ('2024-08', 35), ('2024-09', 34), ('2024-10', 36), ('2024-11', 26), ('2024-12', 41)]


In [83]:
query="""
WITH monthly_trend AS (
    SELECT strftime('%Y-%m', date_created) AS month,
           COUNT(*) AS approved_applications
    FROM application
    WHERE state = 'APPROVED'
      AND strftime('%Y-%m', date_created) >= strftime('%Y-%m', 'now', '-12 months')
      AND office_code IN (SELECT office_code FROM Office WHERE location_code = 'L1')
    GROUP BY month
)
SELECT all_months.month, COALESCE(approved_applications, 0) AS approved_applications
FROM (
    SELECT strftime('%Y-%m', 'now', '-12 months') AS month
    UNION ALL
    SELECT strftime('%Y-%m', 'now', '-11 months')
    UNION ALL
    SELECT strftime('%Y-%m', 'now', '-10 months')
    UNION ALL
    SELECT strftime('%Y-%m', 'now', '-9 months')
    UNION ALL
    SELECT strftime('%Y-%m', 'now', '-8 months')
    UNION ALL
    SELECT strftime('%Y-%m', 'now', '-7 months')
    UNION ALL
    SELECT strftime('%Y-%m', 'now', '-6 months')
    UNION ALL
    SELECT strftime('%Y-%m', 'now', '-5 months')
    UNION ALL
    SELECT strftime('%Y-%m', 'now', '-4 months')
    UNION ALL
    SELECT strftime('%Y-%m', 'now', '-3 months')
    UNION ALL
    SELECT strftime('%Y-%m', 'now', '-2 months')
    UNION ALL
    SELECT strftime('%Y-%m', 'now', '-1 months')
) AS all_months
LEFT JOIN monthly_trend ON all_months.month = monthly_trend.month;


"""

run_query(query=query)

[('2023-04', 34), ('2023-05', 33), ('2023-06', 32), ('2023-07', 34), ('2023-08', 32), ('2023-09', 35), ('2023-10', 37), ('2023-11', 31), ('2023-12', 32), ('2024-01', 36), ('2024-02', 28), ('2024-03', 22)]


In [84]:
query =""" 
SELECT 
    strftime('%Y-%m', date_processed) AS month, 
    COUNT(*) as approved_applications
FROM application
WHERE state = 'APPROVED' AND office_code IN (
    SELECT office_code FROM Office WHERE location_code in ('L1','L2','L3','L4','L5')
) AND date_processed >= DATE('now', '-12 months')
GROUP BY strftime('%Y-%m', date_processed)
ORDER BY month;
"""

run_query(query=query)

[('2023-04', 25), ('2023-05', 170), ('2023-06', 157), ('2023-07', 153), ('2023-08', 160), ('2023-09', 158), ('2023-10', 165), ('2023-11', 159), ('2023-12', 175), ('2024-01', 186), ('2024-02', 158), ('2024-03', 148), ('2024-04', 148), ('2024-05', 154), ('2024-06', 166), ('2024-07', 158), ('2024-08', 166), ('2024-09', 156), ('2024-10', 164), ('2024-11', 149), ('2024-12', 170)]
